# Opgave om Titanic - Martin Patrong Haspang

## Formål: at undersøge om hvorvidt kaptajnen og besætningen på Titanic gjorde hvad de kunne for at redde kvinder og børn først uafhængigt af passagerklasse, eller om 1. klasses (og evt. 2. klasses) passagerer fik forrang til redningsbådene.

### Generelle overvejelser om den anvendte model findes neders i denne notebook.

--------

------------

### Først indlæses de pakker, der skal bruges i det følgende:

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

### Datasættene indlæses:

In [2]:
#pd.read_csv('datasæt del1.csv', sep='\t', lineterminator='^M')
data_part1_raw = pd.read_csv('datasæt del1.csv', sep=',')
data_part2_raw = pd.read_csv('datasæt del2 komma.csv', sep=',')
data_names = pd.read_csv('navne.txt', sep=',')

### Datasættenes typer undersøges:

In [3]:
data_part1_raw.dtypes

pclass        int64
survived    float64
sex          object
age          object
sibsp         int64
parch         int64
ticket       object
fare        float64
cabin        object
embarked     object
boat         object
body        float64
homedest     object
case_id       int64
dtype: object

In [4]:
data_part2_raw.dtypes

pclass        int64
survived      int64
sex          object
age         float64
sibsp         int64
parch         int64
ticket       object
fare         object
cabin        object
embarked     object
boat         object
body        float64
homedest     object
case_id       int64
dtype: object

In [5]:
data_names.dtypes

case_id     int64
name       object
dtype: object

### Det kan ses, at ikke alle variable er deklareret hesigtsmæssigt. Det kan være fordi der er ugyldige værdier. Der vil blive tjekket for ugyldige værdier i databasedelen længere nede (dette kan også gøres i Python inden data indsættes i databasen).

In [6]:
#data_part1_raw = data_part1_raw.astype({"survived": int})

### Datasættenes indhold undersøges:

In [7]:
data_part1_raw.head(5)

,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,homedest,case_id
0,1,0.0,male,67,1,0,PC 17483,221.7792,C55 C57,S,NaN,96.0,"New York, NY",1
1,1,0.0,male,46,0,0,694,26.0000,NaN,S,NaN,80.0,"Bennington, VT",2
2,1,0.0,male,50,1,0,PC 17761,106.4250,C86,C,NaN,62.0,"Deephaven, MN / Cedar Rapids, IA",3
3,1,0.0,male,61,0,0,36963,32.3208,D50,S,NaN,46.0,"Haddenfield, NJ",4
4,1,0.0,male,"32,5",0,0,113503,211.5000,C132,C,NaN,45.0,NaN,5


In [8]:
data_part2_raw.head(5)

,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,homedest,case_id
0,2,0,male,57.0,0,0,219533,12.35,NaN,Q,NaN,NaN,"Glasgow / Bangor, ME",601
1,2,1,male,34.0,0,0,248698,13,D56,S,13,NaN,London,602
2,2,1,female,36.0,1,0,226875,26,NaN,S,11,NaN,"Warwick, England",603
3,3,0,male,37.0,2,0,3101276,7.925,NaN,S,NaN,98.0,"Ruotsinphytaa, Finland New York, NY",604
4,3,0,male,22.5,0,0,2698,7.225,NaN,C,NaN,9.0,NaN,605


In [9]:
data_names.head(5)

,case_id,name
0,1,"Straus, Mr. Isidor"
1,2,"Jones, Mr. Charles Cresson"
2,3,"Douglas, Mr. Walter Donald"
3,4,"Sutton, Mr. Frederick"
4,5,"Keeping, Mr. Edwin"


### Databasen oprettes:

In [10]:
connector = sqlite3.connect('titanic.sqlite')
curs = connector.cursor()

In [11]:
### Hvis databasen skal nulstilles køres følgende:
curs.execute('DROP TABLE passenger_data')
curs.execute('DROP TABLE passenger_names')
curs.execute('DROP TABLE names_join_data')
curs.execute('DROP TABLE names_cleaned')
curs.execute('DROP TABLE null_age')
connector.commit()

### Tabel med data for passagererne oprettes:

In [12]:
curs.execute('CREATE TABLE IF NOT EXISTS passenger_data(pclass INTEGER, survived INTEGER, sex STRING, age REAL, sibsp INTEGER, parch INTEGER, ticket STRING, fare REAL, cabin STRING, embarked STRING, boat INTEGER, body REAL, homedest STRING, case_id INTEGER)')
connector.commit()

### Der indsættes data fra hhv. datasættets del 1 og del 2:

In [13]:
curs.executemany('INSERT INTO passenger_data (pclass, survived, sex, age, sibsp, parch, ticket, fare, cabin, embarked, boat, body, homedest, case_id)' 'VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)', data_part1_raw.values.tolist())
connector.commit()

In [14]:
curs.executemany('INSERT INTO passenger_data (pclass, survived, sex, age, sibsp, parch, ticket, fare, cabin, embarked, boat, body, homedest, case_id)' 'VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)', data_part2_raw.values.tolist())
connector.commit()

### Det tjekkes om databasen har fået det rigtige indhold:

In [15]:
curs.execute('SELECT * FROM passenger_data')
result_passenger_data = pd.DataFrame(curs.fetchall())
connector.commit()

In [16]:
result_passenger_data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,0.0,male,67,1,0,PC 17483,221.779,C55 C57,S,None,96.0,"New York, NY",1
1,1,0.0,male,46,0,0,694,26,None,S,None,80.0,"Bennington, VT",2
2,1,0.0,male,50,1,0,PC 17761,106.425,C86,C,None,62.0,"Deephaven, MN / Cedar Rapids, IA",3
3,1,0.0,male,61,0,0,36963,32.3208,D50,S,None,46.0,"Haddenfield, NJ",4
4,1,0.0,male,"32,5",0,0,113503,211.5,C132,C,None,45.0,None,5


In [17]:
curs.execute('SELECT count(case_id) FROM passenger_data')
case_ids = curs.fetchall()
connector.commit()
case_ids

[(1313,)]

### Der oprettes nu en tabel til passagerernes navne:

In [18]:
curs.execute('CREATE TABLE IF NOT EXISTS passenger_names(name_id INTEGER, name STRING)')
connector.commit()

### Der indsættes data:

In [19]:
curs.executemany('INSERT INTO passenger_names (name_id, name)' 'VALUES (?,?)', data_names.values.tolist())
connector.commit()

### Det undersøges om data er kommet korrekt ind i databasen:

In [20]:
curs.execute('SELECT * FROM passenger_names')
result_passenger_names = pd.DataFrame(curs.fetchall())
connector.commit()

In [21]:
result_passenger_names.head(5)

,0,1
0,1,"Straus, Mr. Isidor"
1,2,"Jones, Mr. Charles Cresson"
2,3,"Douglas, Mr. Walter Donald"
3,4,"Sutton, Mr. Frederick"
4,5,"Keeping, Mr. Edwin"


In [22]:
curs.execute('SELECT count(case_id) FROM passenger_data')
name_ids = curs.fetchall()
connector.commit()
name_ids

[(1313,)]

### Datasættet med navne sammenflettes nu med datasættet med passagerdata:

In [23]:
curs.execute('CREATE TABLE names_join_data AS SELECT passenger_names.*, passenger_data.* FROM passenger_names INNER JOIN passenger_data ON passenger_data.case_id = passenger_names.name_id ORDER BY case_id ASC')
result_all = pd.DataFrame(curs.fetchall())
connector.commit()

### Det kontrolleres, at data er kommet rigtigt ind i den nye tabel:

In [24]:
curs.execute('SELECT * FROM names_join_data')
result_all = pd.DataFrame(curs.fetchall())
connector.commit()

In [25]:
result_all.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,"Straus, Mr. Isidor",1,0.0,male,67,1,0,PC 17483,221.779,C55 C57,S,None,96.0,"New York, NY",1
1,2,"Jones, Mr. Charles Cresson",1,0.0,male,46,0,0,694,26,None,S,None,80.0,"Bennington, VT",2
2,3,"Douglas, Mr. Walter Donald",1,0.0,male,50,1,0,PC 17761,106.425,C86,C,None,62.0,"Deephaven, MN / Cedar Rapids, IA",3
3,4,"Sutton, Mr. Frederick",1,0.0,male,61,0,0,36963,32.3208,D50,S,None,46.0,"Haddenfield, NJ",4
4,5,"Keeping, Mr. Edwin",1,0.0,male,"32,5",0,0,113503,211.5,C132,C,None,45.0,None,5


In [26]:
curs.execute('SELECT count(case_id) FROM names_join_data')
ids = curs.fetchall()
connector.commit()
ids

[(1315,)]

### Det kan ses, at der er kommet flere rækker i den sammenflettede tabel. Det kan skyldes, at der er nogle id-numre, der er blevet indtastet flere gange.

In [27]:
curs.execute('SELECT count(case_id), * FROM names_join_data GROUP BY case_id ORDER BY count(case_id) DESC')
group_by_id = pd.DataFrame(curs.fetchall())
connector.commit()
group_by_id.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,4,550,"Cunningham, Mr. Alfred Fleming",2,0.0,male,None,0,0,239853,0,None,S,None,NaN,Belfast,550
1,1,1,"Straus, Mr. Isidor",1,0.0,male,67,1,0,PC 17483,221.779,C55 C57,S,None,96.0,"New York, NY",1
2,1,2,"Jones, Mr. Charles Cresson",1,0.0,male,46,0,0,694,26,None,S,None,80.0,"Bennington, VT",2
3,1,3,"Douglas, Mr. Walter Donald",1,0.0,male,50,1,0,PC 17761,106.425,C86,C,None,62.0,"Deephaven, MN / Cedar Rapids, IA",3
4,1,4,"Sutton, Mr. Frederick",1,0.0,male,61,0,0,36963,32.3208,D50,S,None,46.0,"Haddenfield, NJ",4


### Det ses, at id-nummer 550 fremkommer 4 gange.

### Datasættet undersøges nu for outliers og fejlbehæftet indhold. Det gøres ad flere omgange:

In [28]:
curs.execute('SELECT * FROM names_join_data ORDER BY survived ASC')
result_survived = pd.DataFrame(curs.fetchall())
connector.commit()
result_survived.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,325,"Wonder, Mr. Stevie",1,NaN,male,68,0,0,36010,10,None,S,None,NaN,"New York City, NY",325
1,1,"Straus, Mr. Isidor",1,0.0,male,67,1,0,PC 17483,221.779,C55 C57,S,None,96.0,"New York, NY",1
2,2,"Jones, Mr. Charles Cresson",1,0.0,male,46,0,0,694,26,None,S,None,80.0,"Bennington, VT",2
3,3,"Douglas, Mr. Walter Donald",1,0.0,male,50,1,0,PC 17761,106.425,C86,C,None,62.0,"Deephaven, MN / Cedar Rapids, IA",3
4,4,"Sutton, Mr. Frederick",1,0.0,male,61,0,0,36963,32.3208,D50,S,None,46.0,"Haddenfield, NJ",4


In [29]:
curs.execute('SELECT * FROM names_join_data ORDER BY survived DESC')
result_survived_desc = pd.DataFrame(curs.fetchall())
connector.commit()
result_survived_desc.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,36,"Frolicher, Miss. Hedwig Margaritha",1,1.0,female,22,0,2,13568,49.5,B39,C,5,NaN,"Zurich, Switzerland",36
1,37,"Hogeboom, Mrs. John C (Anna Andrews)",1,1.0,female,51,1,0,13502,77.9583,D11,S,10,NaN,"Hudson, NY",37
2,38,"Geiger, Miss. Amalie",1,1.0,female,35,0,0,113503,211.5,C130,C,4,NaN,None,38
3,39,"Harder, Mrs. George Achilles (Dorothy Annan)",1,1.0,female,25,1,0,11765,55.4417,E50,C,5,NaN,"Brooklyn, NY",39
4,40,"Taussig, Miss. Ruth",1,1.0,female,18,0,2,110413,79.65,E68,S,8,NaN,"New York, NY",40


In [30]:
curs.execute('SELECT * FROM names_join_data WHERE sex != "male" AND sex != "female"')
result_sex = pd.DataFrame(curs.fetchall())
connector.commit()
result_sex.head(5)

""


In [31]:
curs.execute('SELECT * FROM names_join_data ORDER BY age ASC')
result_age = pd.DataFrame(curs.fetchall())
connector.commit()
result_age.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,62,"Flegenheim, Mrs. Alfred (Antoinette)",1,1.0,female,None,0,0,PC 17598,31.6833,None,S,7,NaN,"New York, NY",62
1,65,"Chibnall, Mrs. (Edith Martha Bowerman)",1,1.0,female,None,0,1,113505,55,E33,S,6,NaN,"St Leonards-on-Sea, England Ohio",65
2,74,"Thorne, Mrs. Gertrude Maybelle",1,1.0,female,None,0,0,PC 17585,79.2,None,C,D,NaN,"New York, NY",74
3,79,"Rheims, Mr. George Alexander Lucien",1,1.0,male,None,0,0,PC 17607,39.6,None,S,A,NaN,"Paris / New York, NY",79
4,98,"Chisholm, Mr. Roderick Robert Crispin",1,0.0,male,None,0,0,112051,0,None,S,None,NaN,"Liverpool, England / Belfast",98


In [32]:
curs.execute('SELECT * FROM names_join_data ORDER BY age DESC')
result_age_desc = pd.DataFrame(curs.fetchall())
connector.commit()
result_age_desc.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,25,"Partner, Mr. Austen",1,0.0,male,"45,5",0,0,113043,28.5,C124,S,None,166.0,"Surbiton Hill, Surrey",25
1,352,"Navratil, Mr. Michel (""Louis M Hoffman"")",2,0.0,male,"36,5",0,2,230080,26,F2,S,None,15.0,"Nice, France",352
2,5,"Keeping, Mr. Edwin",1,0.0,male,"32,5",0,0,113503,211.5,C132,C,None,45.0,None,5
3,331,"Nasser, Mr. Nicholas",2,0.0,male,"32,5",1,0,237736,30.0708,None,C,None,43.0,"New York, NY",331
4,549,"Webber, Miss. Susan",2,1.0,female,"32,5",0,0,27267,13,E101,S,12,NaN,"England / Hartford, CT",549


In [33]:
curs.execute('SELECT * FROM names_join_data WHERE age <=0 ')
result_age_neg = pd.DataFrame(curs.fetchall())
connector.commit()
result_age_neg.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,172,"DiCaprio, Mr. Leonardo",1,1,male,-62.0,0,0,24161,10.0,Shutter Island,S,None,None,"Los Angeles, CA",172


In [34]:
curs.execute('SELECT count(name) FROM names_join_data WHERE survived=0 OR survived=1 ')
total_casualties = curs.fetchall()
connector.commit()
total_casualties

[(1314,)]

### ... der kan laves mange flere queries for at tjekke om indholdet af datasættet er meningsfyldt. Jeg vil ikke skrive det hele her, men opsummere nogle ovservationer:

### Observationer:

- Der er en vis Leonardo Di Caprio om bord med negativ alder
- Stevie Wonder er med, men der er NaN i survived
- Mange af børnene har ikke fået registreret deres alder, men man kan gætte på at det er børn ved at se på parch og sibsp og se på forældrenes alder.
- Der er ikke negative: billetpriser, sibsp, parch.
- Der er et par billetpriser over 1000 - ikke noget mystisk
- Nogle af dem, der har ikke-heltallig alder har komma så decimal-separator, og de registreres som værende over 100 år gamle. Deriblandt en del børn, da de måske er registreret ved et antal måneder - dvs. en brøkdel af et år.
- 265 personer har "None" i alder.
- I familier vil forældre typisk hedde Mr og Mrs, mens børn vil hedde Miss og Master
- Kolonner, der med fordel kan droppes: Ticket number, cabin, embarked, homedest
- Nogle integers er decimaltal - f.eks. survived
- Der er et id-nummer der figurerer to gange i hvert datasæt.

### Det kan ikke garanteres, at alle anomalier er blevet fanget. Hvis f.eks. Leonardo Di Caprios alder havde været positiv, ville det ikke være muligt at opdage ham uden et manuelt eftersyn af navne.

### Datasættet renses nu i henhold til ovenstående. 

### Først laves en ny tabel til de rensede data. Ticket number, cabin, embarked, homedest droppes, resten medtages.

In [35]:
curs.execute('CREATE TABLE names_cleaned AS SELECT name, pclass, survived, sex, age, sibsp, parch, fare, case_id FROM names_join_data ')
connector.commit()

### Rækker der mangler oplysninger om overlevelse droppes (sorry, mr. Wonder...):

In [36]:
curs.execute('DELETE FROM names_cleaned WHERE survived IS NULL ')
connector.commit()

### En enkelt række har et punktum i stedet for en billetpris:

In [37]:
curs.execute('DELETE FROM names_cleaned WHERE fare ="." ')
connector.commit()

### Her mister Leonardo sin plads på båden:

In [38]:
curs.execute('DELETE FROM names_cleaned WHERE age < 0 ')
connector.commit()

### De steder, hvor der er et , som decimaltalsseperator i stedet for . ændres:

In [39]:
curs.execute('UPDATE names_cleaned SET age = REPLACE(age, ",",".")')
connector.commit()

### De rækker, der mangler alder, droppes. Man kunne godt finde frem til hvem der er børn for nogle af rækkerne, men det vil jeg ikke gøre i denne omgang. Rækkerne gemmes i en ny tabel for evt. senere brug.

In [40]:
curs.execute('CREATE TABLE null_age AS SELECT name, pclass, survived, sex, age, sibsp, parch, fare, case_id FROM names_cleaned WHERE age IS NULL ')
connector.commit()

In [41]:
curs.execute('DELETE FROM names_cleaned WHERE age IS NULL')
connector.commit()

In [42]:
curs.execute('SELECT * FROM names_cleaned ORDER BY age ASC')
result_cleaned = pd.DataFrame(curs.fetchall())
connector.commit()

In [43]:
result_cleaned.head(5)

,0,1,2,3,4,5,6,7,8
0,"Dean, Miss. Elizabeth Gladys ""Millvina""",3,1,female,0.1667,1,2,20.5750,1142
1,"Danbom, Master. Gilbert Sigvard Emanuel",3,0,male,0.3333,0,2,14.4000,1081
2,"Thomas, Master. Assad Alexander",3,1,male,0.4167,0,1,8.5167,977
3,"Hamalainen, Master. Viljo",2,1,male,0.6667,1,1,14.5000,479
4,"Baclini, Miss. Helene Barbara",3,1,female,0.7500,2,1,19.2583,799


### Der tilføjes nu en kolonne med en binær variabel, som angiver om man er mand (1) eller kvinde/barn (0):

In [44]:
curs.execute('ALTER TABLE names_cleaned ADD COLUMN ismale INT')
connector.commit()

In [45]:
curs.execute('UPDATE names_cleaned SET ismale = CASE WHEN sex = "male" AND age >= 18 THEN 1 ELSE 0 END')
connector.commit()

In [46]:
curs.execute('SELECT * FROM names_cleaned')
result_all = pd.DataFrame(curs.fetchall())
connector.commit()

In [47]:
result_all.head(5)

,0,1,2,3,4,5,6,7,8,9
0,"Straus, Mr. Isidor",1,0,male,67.0,1,0,221.7792,1,1
1,"Jones, Mr. Charles Cresson",1,0,male,46.0,0,0,26.0000,2,1
2,"Douglas, Mr. Walter Donald",1,0,male,50.0,1,0,106.4250,3,1
3,"Sutton, Mr. Frederick",1,0,male,61.0,0,0,32.3208,4,1
4,"Keeping, Mr. Edwin",1,0,male,32.5,0,0,211.5000,5,1


### Nu kan der udtrækkes oplysninger om hyppigheden for dødsfald som funktion af køn:

In [48]:
curs.execute('SELECT COUNT(name) FROM names_cleaned')
total = curs.fetchall()[0][0]
connector.commit()
total

1046

In [49]:
curs.execute('SELECT COUNT(name) FROM names_cleaned WHERE ismale = 1')
total_men = curs.fetchall()[0][0]
connector.commit()
total_men

575

In [50]:
curs.execute('SELECT COUNT(name) FROM names_cleaned WHERE ismale = 0')
total_women_chld = curs.fetchall()[0][0]
connector.commit()
total_women_chld

471

In [51]:
curs.execute('SELECT COUNT(name) FROM names_cleaned WHERE ismale = 0 AND survived = 1')
women_children_survived = curs.fetchall()[0][0]
connector.commit()
women_children_survived

324

In [52]:
curs.execute('SELECT COUNT(name) FROM names_cleaned WHERE ismale = 0 AND survived = 0')
women_children_dead = curs.fetchall()[0][0]
connector.commit()
women_children_dead

147

In [53]:
curs.execute('SELECT COUNT(name) FROM names_cleaned WHERE ismale = 1 AND survived = 0')
men_dead = curs.fetchall()[0][0]
connector.commit()
men_dead

471

In [54]:
curs.execute('SELECT COUNT(name) FROM names_cleaned WHERE ismale = 1 AND survived = 1')
men_survived = curs.fetchall()[0][0]
connector.commit()
men_survived

104

In [55]:
sex_matrix = np.array([["men", "women+children"], ["dead",men_dead/total_men,women_children_dead/total_women_chld], ["survived",men_survived/total_men,women_children_survived/total_women_chld]])
sex_matrix

array([list(['men', 'women+children']),
       list(['dead', 0.8191304347826087, 0.31210191082802546]),
       list(['survived', 0.1808695652173913, 0.6878980891719745])],
      dtype=object)

### Og tilsvarende som funktion af billetklasse:

In [56]:
curs.execute('SELECT COUNT(name) FROM names_cleaned WHERE Pclass = 1')
total_first = curs.fetchall()[0][0]
connector.commit()
total_first

285

In [57]:
curs.execute('SELECT COUNT(name) FROM names_cleaned WHERE Pclass = 2')
total_second = curs.fetchall()[0][0]
connector.commit()
total_second

261

In [58]:
curs.execute('SELECT COUNT(name) FROM names_cleaned WHERE Pclass = 3')
total_third = curs.fetchall()[0][0]
connector.commit()
total_third

500

In [59]:
curs.execute('SELECT COUNT(name) FROM names_cleaned WHERE Pclass = 1 AND survived = 0')
first_dead = curs.fetchall()[0][0]
connector.commit()
first_dead

103

In [60]:
first_survived = total_first - first_dead
first_survived

182

In [61]:
curs.execute('SELECT COUNT(name) FROM names_cleaned WHERE Pclass = 2 AND survived = 0')
second_dead = curs.fetchall()[0][0]
connector.commit()
second_dead

146

In [62]:
second_survived = total_second - second_dead
second_survived

115

In [63]:
curs.execute('SELECT COUNT(name) FROM names_cleaned WHERE Pclass = 3 AND survived = 0')
third_dead = curs.fetchall()[0][0]
connector.commit()
third_dead

369

In [64]:
third_survived = total_third - third_dead
third_survived

131

In [65]:
class_matrix = np.array([["","first","second","third"],["dead",first_dead/total_first,second_dead/total_second,third_dead/total_third], ["survived",first_survived/total_first,second_survived/total_second,third_survived/total_third]])
class_matrix

array([['', 'first', 'second', 'third'],
       ['dead', '0.36140350877192984', '0.5593869731800766', '0.738'],
       ['survived', '0.6385964912280702', '0.44061302681992337', '0.262']],
      dtype='<U19')

### Der kan på samme vis undersøges sammenhænge mellem andre parametre

### Observationer:
- Det tyder på, at der er meget større sandsynlighed for at overleve, hvis man er kvinde eller barn
- Det tyder på, at der er meget større sandsynlighed for at overleve, jo højere billetklasse man har
- Der er lidt større sandsynlighed for at overleve hvis man er kvinde eller barn på 3. klasse end hvis man er man på 1. klasse. Dette tyder på, at køn har større indflydelse end klasse.
- Der er større sandsynlighed for at overleve hvis man er kvinde end hvis man er barn.
- Der er lidt flere børn på 3. klasse end på 1.
- Der er lidt flere kvinder på 1. klasse end på 3.
- Rangliste over overlevelsesfrekvenser:
- - Kvinder/børn på 1. klasse
- - Kvinder/børn på 2. klasse
- - Kvinder/børn på 3. klasse
- - Mænd på 1. klasse
- - Mænd på 3. klasse
- - Mænd på 2. klasse

### For at undersøge hvilke parametre der har størst indflydelse på sandsynligheden for at overleve, laves en logistisk regression til data.

### Først laves et tilpasset datasæt. Da vi her er interesserede i det bedst mulige fit, snarere end en god forudsigelse, bruges hele datasættet til at fitte den logistiske model.

In [66]:
curs.execute('SELECT * FROM names_cleaned')
train = pd.DataFrame(curs.fetchall())
connector.commit()
train.head(5)

,0,1,2,3,4,5,6,7,8,9
0,"Straus, Mr. Isidor",1,0,male,67.0,1,0,221.7792,1,1
1,"Jones, Mr. Charles Cresson",1,0,male,46.0,0,0,26.0000,2,1
2,"Douglas, Mr. Walter Donald",1,0,male,50.0,1,0,106.4250,3,1
3,"Sutton, Mr. Frederick",1,0,male,61.0,0,0,32.3208,4,1
4,"Keeping, Mr. Edwin",1,0,male,32.5,0,0,211.5000,5,1


### Kolonnerne med navne droppes, da de antages at være uden indflydelse, og kolonnen med køn droppes, da den er blevet erstattet af kolonnen kvinder/børn eller mænd. Derudover droppes kolonnen med id-numre. 

In [67]:
train.drop([0, 3, 8], inplace=True, axis=1)

In [68]:
train.head(5)

,1,2,4,5,6,7,9
0,1,0,67.0,1,0,221.7792,1
1,1,0,46.0,0,0,26.0000,1
2,1,0,50.0,1,0,106.4250,1
3,1,0,61.0,0,0,32.3208,1
4,1,0,32.5,0,0,211.5000,1


### Prediktorerne er nu de resterende variable, pånær "survived", da det er den der skal modelleres:

In [69]:
predictors = train.drop(2, axis=1)

### Der laves et standardiseret sæt af prediktorer:

In [70]:
scaler = StandardScaler()

In [71]:
predictors_std = scaler.fit_transform(predictors)

/home/haalle/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/haalle/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Der advares om, at integers er blevet konverteret til floats. Det er acceptabelt.

### Target-variablen er den binære "survived"-variabel:

In [72]:
target = train[2]

### Den logistiske model initieres:

In [73]:
model = LogisticRegression(solver='liblinear', random_state=0)

### Der laves et fit:

In [74]:
model.fit(predictors_std,target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

### Fit-parametrene angives:

In [75]:
model.intercept_

array([-0.49685131])

In [76]:
model.coef_

array([[-0.92403876, -0.23803958, -0.40652806, -0.02223565,  0.47835892,
        -1.32763581]])

### Fortolkninger:
- Den numerisk største koefficient hører til kvinder/børn (0) eller mænd (1) parameteren. Det betyder, at den har størst indflydelse på sandsynligheden for at overleve. Den korrelerer negativt, så hvis parameteren er høj (1), er sandsynligheden for overlevelse lav.
- Den numerisk næststørste koefficient hører til billetklasse. Den har altså næststørst indflydelse. Den korrelerer negativt, så jo større værdi (dvs. lavere klasse), jo mindre sandsynlighed for overlevelse.
- Dernæst kommer billetprisen. Jo højere pris, jo højere sandsynlighed for at overleve. Denne er næppe uafhængig af klassen.
- Alder korrelerer negativt. Dvs. jo højere alder, jo mindre sandsynlighed for at overleve.
- Søskende/forældre korrelerer negativt. Jo flere søskende/børn/forældre, jo mindre sandsynlighed for at overleve.

#### Det meste er i overensstemmelse med det forventede, men sidstnævne er måske lidt overraskende. Måske kan det skyldes, at hvis den ene i en famile døde, så gjorde den anden det sandsynligvis også, og derfor er dem med familie overrepræsenteret i statistikken.

In [77]:
#model.predict_proba(predictors)
#model.predict(predictors)
#print(classification_report(target, model.predict(predictors)))

In [78]:
model.score(predictors,target)

0.6653919694072657

In [79]:
confusion_matrix(target, model.predict(predictors))

array([[433, 185],
       [165, 263]])

### Overvejelser:
- Rens af data: Der er ikke nogen garanti for, at alle fejl i datasættet er fundet.
- Der er en stor andel af børn i de data, der har Null i alder. Det kan forbedre modellen, hvis man finder frem til børnene i den del af data, der er sorteret fra.
- Parametrenes indflydelse på hinanden:<br>
Der kan være mange måder, hvorpå parametrene afhænger af hinanden. F.eks. er der flere børn på tredje klasse, end på 1., så måske er der overdødelighed på tredje klasse, fordi der er mange børn på tredje klasse. Omvendt kan det også være, at der er stor overdødelighed på tredje klasse, fordi de allesammen er børn. Man kan godt lave en chi^2 test på, om der er signifikant flere, der er døde blandt f.eks. mænd end kvinder/børn. Den vil give en meget stor signifikans på dette, men det er ikke det mest interessante at undersøge. Det er vigtigere at undersøge, om sandsynligheden for at overleve afhænger mest af om man er barn eller af om man er på 3. klasse. Dette kan gøres vha. en logistisk regression, som er en almindelig metode til at håndtere data med mange variable og binært respons.
- Modellen:<br>
Ofte vil man gøre det, at man splitter datasættet i to dele, hvor man bruger første del til sit fit (træne modellen) og anden del til validering af forudsigelserne. I denne sammenhæng er det imidlertid ikke så interessant at kunne forudsige hvem der overlever, hvis der sker et forlis igen, men mere at få et så godt fit som muligt. Derfor er hele datasættet blevet brugt.<br>
Det ville være optimalt at beregne konfidensintervaller og p-værdier for de beregnede modelparametre. Det kunne f.eks. gøres ved at køre modellen et antal gange på tilfældige samples af datasættet, så middelværdier og spredninger kan estimeres på parametrene. Det er imidlertid ikke implementeret endnu.<br>
Modellen er kørt både på et splittet datasæt, med og uden standardisering af parametrene og med udelukkelse af billetprisen (da den må antages at være temmelig afhængig af billetklassen). Alle disse kørsler giver de samme overordnede resultater, så selvom det ikke er en særlig stringent stresstest af modellen, giver det alligevel en ide om, at modellen giver meningsfyldte resultater.<br>
Modellen giver en forholdsvis lav score (ca. 67% korrekte forudsigelser). Dette er ikke nødvendigvis et problem. Dels er det en probabilistisk model, og selvom en given forudsigelse vil slå ud som enten helt korrekt eller helt forkert, kan det godt være, at den sandsynlighedsmodel, der ligger til grund for resultatet kan være fornuftig. Der kan med andre ord være en stor spredning på modellens score. Et andet - og måske vigtigere - forhold er, at det ikke er en korrekt forudsigelse vi er interesserede i, men snarere en vurdering af de enkelte parametres effekt på sandsynligheden for at overleve.
- Fysiske forhold:<br>
Selvom modellen viser, at der er en ret stor betydning af passagerklassen for sandsynligheden for at overleve, er mere fysiske og eventuelt mere kvalitative observationer nødvendige. Eksempelvis gør det en stor forskel for konklusionen, om folk fra tredje klasse aldrig nåede op til bådene, forde deres pladser var placeret langt væk fra bådene, eller om folk fra tredje klasse kontant er blevet afvist på dækket af besætningen. Der er også et psykologisk element der mangler at blive belyst for at kunne vurdere i hvor høj grad kan man egentlig forvente at kaptajnen og besætningen vil lykkes med at overholde retningslinjer, i en situation hvor der givetvis har været en stor grad af kaos og panik.

### To do:
- Finde børn blandt dem med Null i alder.
- Få kolonnenavne på de dataframes, der kommer som udtræk af databasen.
- Database-joined bør være et right eller left-join, da vi gerne vil have alle de rækker hvor der er data og de tilhørende navne, men også de rækker hvor der er data, men ikke navne.
- Regne konfidensintervaller ud for estimaterne af modelparametrene
- Forbedre modellen